In [1]:
#!pip install scikit-network
#!pip install dgl

In [9]:
import ChebNet
import CORA
import utils

import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

In [10]:
# Reload when files are changed
import importlib
importlib.reload(ChebNet)
importlib.reload(CORA)
importlib.reload(utils)

<module 'utils' from '/Users/alice/Documents/GIT/CayleyNets/utils.py'>

In [11]:
# Checking torch and device

# Torch version
print(torch.__version__)

# Is MPS even available? macOS 12.3+
print(torch.backends.mps.is_available())

# Was the current version of PyTorch built with MPS activated?
print(torch.backends.mps.is_built())

2.1.1
True
True


In [12]:
cora = CORA.CORA()

Parsing files...
Done.


In [13]:
# device = torch.device('mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [14]:
train_mask, test_mask, val_mask = utils.split_train_test_val(cora.n_nodes)

In [15]:
# Hyperparameters
in_feats = cora.n_features
n_classes = cora.n_classes
n_hidden = 16
n_layers = 1 # number of hidden+output layers
k = 5 # Chebyshev polynomial order

In [16]:
# Graph as an object
graph = dgl.from_scipy(cora.adjacency)

# Features and labels as tensors
features = torch.Tensor(cora.features).to(device)
labels = torch.Tensor(cora.labels).long().to(device)

# Masks as tensors
train_mask = torch.Tensor(train_mask).bool().to(device)
test_mask = torch.Tensor(test_mask).bool().to(device)
val_mask = torch.Tensor(val_mask).bool().to(device)

In [17]:
# Define model
model = ChebNet.ChebNet(graph, in_feats, n_classes, n_hidden, n_layers, k, bias=True).to(device)
model

ChebNet(
  (layers): ModuleList(
    (0): ChebConv(
      (linear): Linear(in_features=7165, out_features=16, bias=True)
    )
    (1): ChebConv(
      (linear): Linear(in_features=80, out_features=7, bias=True)
    )
  )
)

In [18]:
print(f'Number of parameters: {sum(p.numel() for p in model.parameters())}')
print(f'Number of parameters: {sum(p.numel() for p in model.layers[0].parameters())}')
print(f'Number of parameters: {sum(p.numel() for p in model.layers[1].parameters())}')

Number of parameters: 115223
Number of parameters: 114656
Number of parameters: 567


In [19]:
# Optimizer
lr = 5e-3
weight_decay = 5e-4
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

In [22]:
def eval_model(model, features, labels, test_mask):
    '''Evaluate the model in terms of accuracy.'''
    model.eval()
    with torch.no_grad():
        output = model(features)
        labels_pred = torch.max(output, dim=1)[1]
        score = np.mean(np.array(labels[test_mask]) == np.array(labels_pred[test_mask]))
    return score

In [21]:
# Train the model
batch_size = 32
epochs = 100

# Save loss values for plotting
loss_values = []
scores = []
for e in range(epochs):
    # Compute output
    output = model(features)

    # Compute loss
    logp = F.log_softmax(output, 1)
    loss = F.nll_loss(logp[train_mask], labels[train_mask])
    loss_values.append(loss.item())

    # Perform backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Evaluation
    score = eval_model(model, graph, features, labels, val_mask)
    scores.append(score)

    # Print loss
    print(f'Epoch: {e:03d} | Loss: {loss:.4f}')

Epoch: 000 | Loss: 1.9454
Epoch: 001 | Loss: 1.8889
Epoch: 002 | Loss: 1.8144
Epoch: 003 | Loss: 1.7448
Epoch: 004 | Loss: 1.6902
Epoch: 005 | Loss: 1.6250
Epoch: 006 | Loss: 1.5478
Epoch: 007 | Loss: 1.4789
Epoch: 008 | Loss: 1.4212
Epoch: 009 | Loss: 1.3689
Epoch: 010 | Loss: 1.3227
Epoch: 011 | Loss: 1.2853
Epoch: 012 | Loss: 1.2566
Epoch: 013 | Loss: 1.2294
Epoch: 014 | Loss: 1.2030
Epoch: 015 | Loss: 1.1796
Epoch: 016 | Loss: 1.1607
Epoch: 017 | Loss: 1.1443
Epoch: 018 | Loss: 1.1284
Epoch: 019 | Loss: 1.1136
Epoch: 020 | Loss: 1.1004
Epoch: 021 | Loss: 1.0881
Epoch: 022 | Loss: 1.0762
Epoch: 023 | Loss: 1.0647
Epoch: 024 | Loss: 1.0538
Epoch: 025 | Loss: 1.0436
Epoch: 026 | Loss: 1.0343
Epoch: 027 | Loss: 1.0213
Epoch: 028 | Loss: 1.0019
Epoch: 029 | Loss: 0.9824
Epoch: 030 | Loss: 0.9673
Epoch: 031 | Loss: 0.9567
Epoch: 032 | Loss: 0.9466
Epoch: 033 | Loss: 0.9336
Epoch: 034 | Loss: 0.9175
Epoch: 035 | Loss: 0.8999
Epoch: 036 | Loss: 0.8824
Epoch: 037 | Loss: 0.8656
Epoch: 038 |

In [23]:
eval_model(model, features, labels, test_mask)

0.7601476014760148